# 딥러닝 안쓰고 주가 예측하기 part2

본 tutorial에선 deep learning을 사용하지 않고 주가를 예측하는 방법에 대하여 다룹니다.

<다룰 내용>
- Stochastic Process
- Geometric Brownian Motion (GBM)
- Monte Carlo simulation

## 준비하기

지난 시간과 마찬가지로 나스닥 지수를 바탕으로 tutorial을 진행합니다.
우선, 같은 방식으로 나스닥 지수 데이터를 준비합니다.

In [ ]:
# 이번 tutorial에서 사용할 파이썬 라이브러리를 불러옵니다.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance
import pandas_datareader as pdr
from datetime import datetime, timedelta, timezone

%matplotlib inline

In [ ]:
# 지금까지의 모든 나스닥 지수(^IXIC)를 가져옵니다.
now = datetime.now(timezone(timedelta(hours=-4)))
df = pdr.get_data_yahoo('^IXIC', start='1971-02-05', end=now.strftime('%Y-%m-%d'))
df = df[['Close']]

# 전일 종가대비 금일 종가의 수익률(return)을 계산합니다.
df['Return'] = df['Close'].pct_change()
df = df.iloc[1:]  # 첫 번째 데이터는 전날 종가가 없으므로 NaN이 나옵니다. 따라서 제거하도록 합니다.

df

## Monte Carlo simulation

어떤 수식을 정확하게 알 수 있다면 해당 수식을 쓰면 되겠지만, 많은 경우엔 정확한 수식을 알 수 없습니다.
따라서 다양한 방식을 이용해 함수를 근사적으로 구하게 됩니다.
예컨대, 딥러닝도 결국 데이터 $X$와 출력 $y$ 사이의 관계를 명확하게 알 수 없기에 $f(X) = y$인 함수 $f$를 근사적으로 구하는 행위입니다.

Monte Carlo simulation[[1]](https://en.wikipedia.org/wiki/Monte_Carlo_method)도 같은 역할을 수행하는 것인데, 반복적인 무작위 샘플링을 이용해 어떤 값을 근사하는 방법입니다. 간단하지만 매우 강력한 도구로, AlphaGo 등의 최신 연구에서도 쓰이고 있습니다. 우선, 반지름이 1인 사분원의 면적을 구하는 예시를 이용해 Monte Carlo simulation을 알아보겠습니다.
(물론 사분원의 면적을 구하는 명확한 공식이 있지만, 이해를 쉽게 돕기 위해 이 예시를 사용합니다.)

사분원의 면적을 Monte Carlo simulation으로 구하는 알고리즘은 $x \in [0, 1]$, $y \in [0, 1]$인 임의의 $x$, $y$를 반복적으로 만들어 전체 점의 개수 중 사분원 내부에 들어오는 점의 개수의 비율로 구합니다.
반지름이 1인 사분원의 면적은 가로 세로가 1인 정사각형에서 사분원이 차지하는 비율과 같으므로, 직관적으로 위 방식이 잘 동작할 것이라 예상할 수 있습니다.

In [ ]:
# N개 만큼의 점을 찍어, 그중 사분원 위에 존재하는 점의 개수를 따로 셉니다.
N = 1000000
in_counter = 0

for i in range(N):
    x, y = np.random.rand(2)
    
    if x ** 2 + y **2 <= 1:  # 사분원의 정의입니다.
        in_counter = in_counter + 1

In [ ]:
# 실제 값과 유사한지 확인해봅시다.
print('with MC:', in_counter/N)
print('정확한 값:', np.pi / 4)  # 사분원의 면적은 πr²/4입니다.

## Stochastic Process

Stochastic Process란 시간의 진행에 대해 확률적인 변화를 가지는 구조를 의미합니다[[2]](https://en.wikipedia.org/wiki/Stochastic_process).
일반적인 확률 변수를 $X(a)$라 한다면, stochastic process는 시간에 대한 변수 $t$를 더하여 $X(a, t)$라 표기합니다.
주식은 시간에 따른 변수이기에 stochastic process로 표현할 수 있습니다.

다양한 stochastic process가 존재하지만, 특히 주식은 기하 브라운 운동(GBM, Geometric Brownian Motion)[[3]](https://en.wikipedia.org/wiki/Geometric_Brownian_motion)이란 stochastic process를 이용해 모델링할 수 있습니다. GBM의 정의는 다음과 같습니다.

$S(t) = S(0) \cdot e^{\left( \mu - \frac{1}{2} \sigma^2 \right)t + \sigma B_t}$
where $S_t$는 시간 $t$에서의 주가, $\mu$는 일일 수익률의 평균, $\sigma$는 일일 수익률의 표준편차, $B_t \sim N(0, 1)$.

미래의 주가는 정확한 함수를 구하기 어려우니, GBM을 이용해 Monte Carlo simulation을 통해 근사할 수 있습니다.
최근 20거래일 정보를 이용해 20 거래일 후의 주가를 예측해봅시다.

In [ ]:
# 최근 20 거래일의 정보를 가져옵니다.
df_latest = df.iloc[-20:]
df_latest

In [ ]:
# GBM을 사용하기 위해 일일 수익률의 평균과 표준편차를 구합니다.
mu = df_latest['Return'].mean()
sigma = df_latest['Return'].std()
print('mu:', mu, 'sigma:', sigma)

In [ ]:
# GBM을 이용해 20일간 주가를 예측하는 함수입니다.
def gbm(start_price):
    B = np.random.randn(20)  # 표준정규분포를 따르는 난수 20개 생성
    ts = np.arange(1, 21)
    expected_return = (mu - 0.5 * (sigma ** 2)) + sigma * B
    
    return start_price * np.exp(expected_return)

In [ ]:
# GBM을 이용한 주가 예측을 그림으로 그려봅시다.
predict = gbm(df_latest['Close'].iloc[-1])

x_axis = np.arange(20)
plt.plot(x_axis, predict)

In [ ]:
# 이번에는 많이 예측하여 평균을 내어봅시다.
N = 1000000
predicts = np.zeros((N, 20))

start_price = df_latest['Close'].iloc[-1]
for i in range(N):
    predicts[i] = gbm(start_price)
    
predict = predicts.mean(axis=0)

print('20 거래일 후 예상 지수: {:.2f}'.format(predict[-1]))
print('20 거래일 후 예상 수익률: {:.2f}%'.format((predict[-1] - start_price) / start_price * 100))

In [ ]:
# 예상 주가를 바탕으로 그래프를 그려봅시다.
plt.ticklabel_format(useOffset=False)  # scientific notation 사용 방지
plt.plot(x_axis, predict)